# reward_model

> Fill in a module description here

In [ ]:
#| default_exp reward

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import torch
from torch import nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from einops import rearrange

from instruct_goose.utils import load_yaml

/Users/education/DATA/projects/ai/RLHF/instructGOOSE/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Reward Model

In [ ]:
# def compute_policy_shift_loss(self, current_policy_dist, old_policy_dist):
#     #     assert current_policy_dist.sum(dim=-1) == 1
#     #     assert old_policy_dist.sum(dim=-1) == 1
        
#     #     kl_loss = nn.KLDivLoss()
        
#     #     return kl_loss(current_policy_dist.log(), old_policy_dist)
    
#     # def compute_reward_value(self, prompt: str):
#     #     inputs = self.tokenizer(prompt, return_tensors="pt")
    
#     # def compute_loss(self):
#     #     loss_policy_shift = self.compute_policy_shift_loss()

In [ ]:
#| export
class RewardModel(nn.Module):
    def __init__(self, checkpoint: str, dropout: float = 0.1):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        self.model = AutoModel.from_pretrained(checkpoint)
        
        config = self.model.config
        n_embed = config.n_embd
        
        # custom head
        self.dropout = nn.Dropout(dropout)
        self.reward_head = nn.Linear(n_embed, 1)
        
    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor):
        last_hidden_state = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
        ).last_hidden_state
        
        output = self.dropout(last_hidden_state)
        output = self.reward_head(output)
                
        # output = rearrange(output, 'b 1 t 1 -> b t')
        # for eacb item in the batch
        # choose the hidden state of the last token as a reward!
        reward_scalar = output[:, -1, 0]
        
        return reward_scalar

In [ ]:
# config = load_yaml("../configs/sentiment_config.yml")

In [ ]:
# checkpoint = config["reward_model"]["model_path"]
# reward_model = RewardModel(checkpoint, dropout=0.1)

### Loss function

$\operatorname{loss}(\theta)=-\frac{1}{\left(\begin{array}{c}
K \\
2
\end{array}\right)} E_{\left(x, y_w, y_l\right) \sim D}\left[\log \left(\sigma\left(r_\theta\left(x, y_w\right)-r_\theta\left(x, y_l\right)\right)\right)\right]$

In [ ]:
#| export
class RewardLoss(nn.Module):
    def forward(self, chosen_rewards: torch.Tensor, rejected_rewards: torch.Tensor):
        assert len(chosen_rewards) == len(rejected_rewards)
        batch_size = len(chosen_rewards)
        
        probs = F.sigmoid(chosen_rewards - rejected_rewards, dim=-1)
        return -probs.mean() / batch_size